In [2]:
import cv2
import numpy as np
from time import sleep
import matplotlib.pyplot as plt

In [2]:
PATH = "/Users/guyperets/Downloads/example_adva_full.mov"
SMALL_CLIP_PATH = "/Users/guyperets/Downloads/SmallClip.mp4"
# Background Subtractor
backSub = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=50, detectShadows=False)

In [5]:
cap = cv2.VideoCapture(PATH)
i = 0
# ? Define a ROI
while True:
    ret, frame = cap.read()
    if ret:
        i += 1
        if i < 200:
            continue
        if i == 200:
            print("Started IP...")
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(frame, (3, 3), 0)
        canny = cv2.Canny(blurred, 80, 240)
        backsub = backSub.apply(canny)
        # _, binary = cv2.threshold(backsub, 100, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        dilated = cv2.dilate(backsub, (5, 5), iterations=10)
        contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        mask = np.zeros_like(frame)
        centroids = []
        for cnt in contours:
            # Connect the contours
            cv2.drawContours(mask, [cnt], -1, (255, 255, 255), cv2.FILLED)
            M = cv2.moments(cnt)
            if M['m00'] != 0:
                cX = int(M['m10'] / M['m00'])
                cY = int(M['m01'] / M['m00'])
                centroids.append((cX, cY))
        # close_centroids = []
        # for c1 in centroids:
        #     for c2 in centroids:
        #         if c1 != c2:
        #             if abs(c1[0] - c2[0]) > 20 and abs(c1[1] - c2[1]) < 20:
        #                 close_centroids.append((c1, c2))
        # for close_pair in close_centroids:
        #     cv2.fillPoly(mask, np.array([close_pair]), (255, 255, 255))
            
        cv2.imshow("BackSub", mask)
        key = cv2.waitKey(0)
        if key & 0xFF == 27 or key & 0xFF == ord('q'): # ESC or 'q'
            break
    else:
        print("Video ended")
        break

cap.release()
cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)



Started IP...


-1

In [ ]:
cap = cv2.VideoCapture(PATH)
backSub = cv2.createBackgroundSubtractorMOG2()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Video ended or failed to capture frame.")
        break

    # Apply background subtraction
    fgMask = backSub.apply(frame)

    # Threshold the foreground mask to get a binary image
    _, binary = cv2.threshold(fgMask, 200, 255, cv2.THRESH_BINARY)

    # Find contours in the binary image
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create an empty mask with the same size as the frame
    mask = np.zeros_like(frame, dtype=np.uint8)

    # Filter and draw contours on the mask
    for cnt in contours:
        if cv2.contourArea(cnt) > 500:  # Filter small contours based on area
            cv2.drawContours(mask, [cnt], -1, (255, 255, 255), thickness=cv2.FILLED)

    # Convert the mask to grayscale and apply threshold to ensure binary mask
    mask_gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    _, mask_binary = cv2.threshold(mask_gray, 1, 255, cv2.THRESH_BINARY)

    # Display the original frame and the mask
    cv2.imshow("Frame", frame)
    cv2.imshow("Mask", mask_binary)

    # Wait for a key press and break the loop if 'q' or 'ESC' is pressed
    key = cv2.waitKey(30) & 0xFF
    if key == 27 or key == ord('q'):
        break
    
cap.release()
cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)


# Test Calibration

In [17]:
def track_object(roi):
    # Initialize the Kalman filter
    kf = cv2.KalmanFilter(4, 2, 0)
    kf.measurementMatrix = np.array([[1, 0], [0, 1]])
    kf.processNoiseCov = np.array([[0.1, 0], [0, 0.1]])
    kf.measurementNoiseCov = np.array([[0.01, 0], [0, 0.01]])

    # Get the current measurement of the object
    x, y = (roi.shape[1] // 2, roi.shape[0] // 2)
    measurement = np.array([x, y])

    # Update the Kalman filter with the current measurement
    kf.predict()
    kf.update(measurement)

    # Predict the next state of the object
    prediction = kf.predict()

    return prediction

In [1]:
def create_multi_tracker(joint_list):
    multi_tracker = cv2.legacy.MultiTracker_create()

    bboxes = []
    colors = [] 
    
    # OpenCV's selectROI function doesn't work for selecting multiple objects in Python
    # So we will call this function in a loop till we are done selecting all objects
    for i in range(len(joint_list)):
        # draw bounding boxes over objects
        # selectROI's default behaviour is to draw box starting from the center
        # when fromCenter is set to false, you can draw box starting from top left corner
        bbox = cv2.selectROI('MultiTracker', frame)
        bboxes.append(bbox)
        colors.append((randint(0, 255), randint(0, 255), randint(0, 255)))
        print("Press q to quit selecting boxes and start tracking")
        print("Press any other key to select next object")


    print(f'Selected bounding boxes: {bboxes}')

    for bbox in bboxes:
        multi_tracker.add(cv2.legacy.TrackerMOSSE_create(), frame, bbox)
        
    
    return multi_tracker

In [1]:
def normalize_frame(frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Normalize the frame to be in the range [0, 1]
    frame = frame.astype(np.float32)
    frame = (frame - frame.min()) / (frame.max() - frame.min())
    return frame

# Test Joint Tracking (Using OpenCV Built-In Trackers)
## Currently Working HERE

In [3]:
import cv2
import numpy as np
from time import sleep
import matplotlib.pyplot as plt
import sys
# from random import randint


cap = cv2.VideoCapture(1)
sleep(.5)
ret, frame = cap.read()
if not ret:
    print("Failed to capture frame.")
    sys.exit()
height, width, _ = frame.shape
print(f'Frame shape: {frame.shape}')
w = 70
h = 70
# joints = {"head": (width // 2 - w // 2, height // 4 - h // 2, 100, 100), # (x, y, w, h) - xy is the top left corner
#           "right_elbow": (width // 2 + w, height // 4 + h, 100, 100),
#           "left_elbow": (width // 2 - 2 * w, height // 4 + h, 100, 100),
#           "torso": (width // 2 - w // 2, height // 4 + h, 100, 100),
#           "right_knee": (width // 2 + int(1.5*w / 2) - 50, height // 4 + 3 * h, 100, 100),
#           "left_knee": (width // 2 - int(1.5*w / 2) - 50, height // 4 + 3 * h, 100, 100)}

joints = {"shoulder": (width // 2 - w // 2, height // 5, w, h), # (x, y, w, h) - xy is the top left corner
          "elbow": (width // 2 + w, height // 5, w, h),
          "wrist": (width // 2 + 2 * w, height // 5, w, h),
          "hip": (width // 2 - w // 2, height // 5 + 2 * h, w, h)}
        #   "knee": (width // 2 - w // 2, height // 5 + 3 * h, w, h),
        #   "ankle": (width // 2 - w // 2, height // 5 + 4 * h + 50, w, h)}
if not ret:
    print("Failed to capture frame.")

# Create a tracker for each joint
trackers = {joint: cv2.legacy.TrackerCSRT_create() for joint in joints.keys()}

for joint, tracker in trackers.items():
    ret = tracker.init(frame, joints[joint])
    if not ret:
        print(f"Failed to initialize tracker for {joint}")

backSub = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=50, detectShadows=False)
i = 0
while True:
    i += 1
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    if not ret:
        print("Failed to capture frame.")
        break
    if i < 500:
        # Apply background subtraction
        # fgMask = backSub.apply(frame)
        # Show each rectangle in the right position
        # cv2.rectangle(frame, joints["head"], (0, 255, 0), 2)
        # cv2.rectangle(frame, joints["right_elbow"], (0, 255, 0), 2)
        # cv2.rectangle(frame, joints["left_elbow"], (0, 255, 0), 2)
        # cv2.rectangle(frame, joints["torso"], (0, 255, 0), 2)
        # cv2.rectangle(frame, joints["right_knee"], (0, 255, 0), 2)
        # cv2.rectangle(frame, joints["left_knee"], (0, 255, 0), 2)
        cv2.rectangle(frame, joints["shoulder"], (0, 255, 0), 2)
        cv2.rectangle(frame, joints["elbow"], (0, 255, 0), 2)
        cv2.rectangle(frame, joints["wrist"], (0, 255, 0), 2)
        cv2.rectangle(frame, joints["hip"], (0, 255, 0), 2)
        # cv2.rectangle(frame, joints["knee"], (0, 255, 0), 2)
        # cv2.rectangle(frame, joints["ankle"], (0, 255, 0), 2)
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1) & 0xFF
        if key == 27 or key == ord('q'):
            break
        continue
    
    centroids = {}
    # success, boxes = multi_tracker.update(frame)
    # print(f'Updated tracker, success = {success}')
    # if success:
    #     for i, box in enumerate(boxes):
    #         print(f'Tracking box {list(joints.keys())[i]}: {box}')
    #         x, y, w, h = [int(v) for v in box]
    #         center_x = x + w // 2
    #         center_y = y + h // 2
    #         cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    #         cv2.circle(frame, (center_x, center_y), 5, (255, 0, 0), -1)
    #         centroids[list(joints.keys())[i]] = (center_x, center_y)
    # else:
    #     cv2.putText(frame, "Tracking failure detected", (100, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
    for joint, tracker in trackers.items():
        success, box = tracker.update(frame)
        if success:
            x, y, w, h = [int(v) for v in box]
            center_x = x + w // 2
            center_y = y + h // 2
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.circle(frame, (center_x, center_y), 5, (255, 0, 0), -1)
            centroids[joint] = (center_x, center_y)
        else:
            cv2.putText(frame, f"Tracking failure detected for {joint}", (100, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
    
    
    # Connect the centroids with lines
    if len(list(centroids.values())) == len(joints.keys()):
        cv2.line(frame, centroids["shoulder"], centroids["elbow"], (0, 255, 0), 2)
        cv2.line(frame, centroids["elbow"], centroids["wrist"], (0, 255, 0), 2)
        # cv2.line(frame, centroids["hip"], centroids["knee"], (0, 255, 0), 2)
        # cv2.line(frame, centroids["knee"], centroids["ankle"], (0, 255, 0), 2)
    cv2.imshow("Frame", frame)
    

    key = cv2.waitKey(1) & 0xFF
    if key == 27 or key == ord('q'):
        break
    
cap.release()
cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)


Frame shape: (720, 1280, 3)


-1

In [1]:
import cv2
import sys

# Set up tracker.
# Instead of MIL, you can also use

tracker_types = ['BOOSTING', 'MIL','KCF', 'TLD', 'MEDIANFLOW', 'DaSiamRPN', 'MOSSE', 'CSRT']
tracker_type = tracker_types[2]
i = 0
while i < len(tracker_types):
    for tracker_type in tracker_types:
        if tracker_type == 'BOOSTING':
            tracker = cv2.legacy.TrackerBoosting_create()
        # if tracker_type == 'MIL':
        #     tracker = cv2.TrackerMIL_create()
        # if tracker_type == 'KCF':
        #     tracker = cv2.TrackerKCF_create()
        elif tracker_type == 'TLD':
            tracker = cv2.legacy.TrackerTLD_create()
        elif tracker_type == 'MEDIANFLOW':
            tracker = cv2.legacy.TrackerMedianFlow_create()
        elif tracker_type == 'MOSSE':
            tracker = cv2.legacy.TrackerMOSSE_create()
        elif tracker_type == "CSRT":
            tracker = cv2.TrackerCSRT_create()
        else:
            continue

        # Read video
        video = cv2.VideoCapture(0)

        # Exit if video not opened.
        if not video.isOpened():
            print("Could not open video")
            sys.exit()

        # Read first frame.
        ok, frame = video.read()
        if not ok:
            print('Cannot read video file')
            sys.exit()
            
        # Define an initial bounding box
        bbox = (287, 23, 86, 320)

        # Uncomment the line below to select a different bounding box
        bbox = cv2.selectROI(frame, False)

        # Initialize tracker with first frame and bounding box
        ok = tracker.init(frame, bbox)

        while True:
            # Read a new frame
            ok, frame = video.read()
            if not ok:
                break
                
            # Start timer
            timer = cv2.getTickCount()

            # Update tracker
            ok, bbox = tracker.update(frame)

            # Calculate Frames per second (FPS)
            fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer)

            # Draw bounding box
            if ok:
                # Tracking success
                p1 = (int(bbox[0]), int(bbox[1]))
                p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
                cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
            else :
                # Tracking failure
                cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)

            # Display tracker type on frame
            cv2.putText(frame, tracker_type + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50),2)
            
            # Display FPS on frame
            cv2.putText(frame, "FPS : " + str(int(fps)), (100,50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (50,170,50), 2)

            # Display result
            cv2.imshow("Tracking", frame)
            
            key = cv2.waitKey(1) & 0xFF
            if key == 27 or key == ord('q'):
                break
        cv2.waitKey(1)
        cv2.destroyAllWindows()
        cv2.waitKey(1)
        i += 1

2024-08-14 16:36:10.008 Python[76304:4961073] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


Cannot read video file


SystemExit: 

/Users/guyperets/Documents/IP_Course/YogaCoach/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
cap = cv2.VideoCapture(0)
ret, frame = cap.read()
height, width, _ = frame.shape
joints = {"head": (int(width * 0.25), int(height * 0.25), int(width * 0.3), int(height * 0.3)),
          "right_elbow": (int(width * 0.6), int(height * 0.6), int(width * 0.2), int(height * 0.3)),
          "left_elbow": (int(width * 0.2), int(height * 0.6), int(width * 0.2), int(height * 0.3))}
if not ret:
    print("Failed to capture frame.")
    exit()
# Initialize the tracker with the bounding box

tracker_head = cv2.legacy.TrackerMOSSE_create()
tracker_right_elbow = cv2.legacy.TrackerMOSSE_create()
tracker_left_elbow = cv2.legacy.TrackerMOSSE_create()

for joint, rect in joints.items():
    x, y, w, h = rect
    match joint:
        case "head":
            tracker_head.init(frame, (x, y, w, h))
        case "right_elbow":
            tracker_right_elbow.init(frame, (x, y, w, h))
        case "left_elbow":
            tracker_left_elbow.init(frame, (x, y, w, h))

backSub = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=50, detectShadows=False)
i = 0
while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    if not ret:
        print("Failed to capture frame.")
        break
    if i < 300:
        # Show each rectangle in the right position
        cv2.rectangle(frame, joints["head"], (0, 255, 0), 2)
        cv2.rectangle(frame, joints["right_elbow"], (0, 255, 0), 2)
        cv2.rectangle(frame, joints["left_elbow"], (0, 255, 0), 2)
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1) & 0xFF
        if key == 27 or key == ord('q'):
            break
        i += 1
        continue
    centroids = {}
    for joint in joints.keys():
        match joint:
            case "head":
                rect = joints[joint]
                height, width, _ = frame.shape
                ret, bbox = tracker_head.update(frame)
                if ret:
                    # Tracking success
                    x, y, w, h = [int(v) for v in bbox]
                    # cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    center_x = x + w // 2
                    center_y = y + h // 2
                    cv2.circle(frame, (center_x, center_y), 5, (255, 0, 0), -1)
                    centroids["head"] = (center_x, center_y)
                else:
                    # Tracking failure
                    cv2.putText(frame, "Tracking failure detected", (100, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
            case "right_elbow":
                rect = joints[joint]
                height, width, _ = frame.shape
                ret, bbox = tracker_right_elbow.update(frame)
                if ret:
                    # Tracking success
                    x, y, w, h = [int(v) for v in bbox]
                    # cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    center_x = x + w // 2
                    center_y = y + h // 2
                    cv2.circle(frame, (center_x, center_y), 5, (255, 0, 0), -1)
                    centroids["right_elbow"] = (center_x, center_y)
                else:
                    # Tracking failure
                    cv2.putText(frame, "Tracking failure detected", (100, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
            case "left_elbow":
                rect = joints[joint]
                height, width, _ = frame.shape
                ret, bbox = tracker_left_elbow.update(frame)
                if ret:
                    # Tracking success
                    x, y, w, h = [int(v) for v in bbox]
                    # cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    center_x = x + w // 2
                    center_y = y + h // 2
                    cv2.circle(frame, (center_x, center_y), 5, (255, 0, 0), -1)
                    centroids["left_elbow"] = (center_x, center_y)
                else:
                    # Tracking failure
                    cv2.putText(frame, "Tracking failure detected", (100, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
    
    # Connect the centroids with lines
    if len(centroids) == 3:
        cv2.line(frame, centroids["head"], centroids["right_elbow"], (0, 255, 0), 2)
        cv2.line(frame, centroids["head"], centroids["left_elbow"], (0, 255, 0), 2)
    cv2.imshow("Frame", frame)
    

    key = cv2.waitKey(1) & 0xFF
    if key == 27 or key == ord('q'):
        break
    
cap.release()
cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

# Test With Adva Plank Video

In [9]:
import cv2
import numpy as np
from time import sleep
import matplotlib.pyplot as plt
import sklearn

In [10]:
CLIP_PATH = "/Users/guyperets/Documents/IP_Course/YogaCoach/adav_example_plank.mov"

In [11]:
def detect_person(frame):
    # back_sub = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=100, detectShadows=False)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(frame, (3, 3), 0)
    canny = cv2.Canny(blurred, 80, 240)
    # cbacksub = back_sub.apply(canny)
    dilated = cv2.dilate(canny, (3, 3), iterations=15)
    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    mask = np.zeros_like(frame)
    centroids = []
    for cnt in contours:
        # Connect the contours
        cv2.drawContours(mask, [cnt], -1, (255, 255, 255), cv2.FILLED)
        M = cv2.moments(cnt)
        if M['m00'] != 0:
            cX = int(M['m10'] / M['m00'])
            cY = int(M['m01'] / M['m00'])
            centroids.append((cX, cY))
    return mask, centroids

In [12]:
def detect_purple_yoga_mat(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower_purple = np.array([125, 60, 60])
    upper_purple = np.array([150, 255, 255])
    mask = cv2.inRange(hsv, lower_purple, upper_purple)
    # Detect the biggest contour and return its bounding box
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) == 0:
        return None
    max_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(max_contour)
    return mask, (x, y, w, h)


In [13]:
def detect_head(binary_frame): # Highest white pixel in the binary image
    # Get the indices of the white pixels
    white_pixels = np.argwhere(binary_frame == 255)
    if len(white_pixels) == 0:
        return None
    head_pixel = white_pixels[np.argmin(white_pixels[:, 0])]
    # print(f"Found top white pixel at {head_pixel}")
    return head_pixel

In [14]:
def detect_foot(binary_roi):
    black_pixels = np.argwhere(binary_roi == 0)
    if len(black_pixels) == 0:
        return None
    # Get the pixel who's closest to the center of the ROI
    center = np.array(binary_roi.shape) // 2
    foot_pixel = black_pixels[np.argmin(np.linalg.norm(black_pixels - center, axis=1))]
    print(f"Found top black pixel at {foot_pixel}")
    # Extract the black pixel from the roi
    cv2.circle(binary_roi, (foot_pixel[1], foot_pixel[0]), 50, 255, -1)
    cv2.imshow("Frame", binary_roi)
    key = cv2.waitKey(0)
    if key == 27 or key == ord('q'):
        cv2.waitKey(1)
        cv2.destroyAllWindows()
        cv2.waitKey(1)
    return foot_pixel

In [2]:
import cv2
def load_frames(path):
    cap = cv2.VideoCapture(path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.flip(frame, 1)
        frames.append(frame)
    return frames

In [16]:
frames = load_frames(CLIP_PATH)

In [12]:
for frame in frames:
    yoga_mat_mask, mat_bbox = detect_purple_yoga_mat(frame)
    x, y, w, h = mat_bbox
    mask, _ = detect_person(frame)
    # Define a ROI in the mat_bbox, and circle every black contour in this ROI
    roi = mask[y:y+h, x:x+w]
    # Draw the biggest contours on the mask according to a threshold
    contours, _ = cv2.findContours(roi, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours:
        if cv2.contourArea(cnt) > 100:
            # Circle the centroid of the contour
            M = cv2.moments(cnt)
            if M['m00'] != 0:
                cX = int(M['m10'] / M['m00'])
                cY = int(M['m01'] / M['m00'])
                cv2.circle(mask, (x + cX, y + cY), 5, (255, 0, 0), -1)
    head = detect_head(mask)
    foot = detect_foot(yoga_mat_mask[y:y+h, x:x+w])
    # For foot: Match the ROI pixel coordinates to the frame coordinates
    foot = (foot[0] + y, foot[1] + x)
    cv2.line(mask, (head[1], head[0]), (foot[1], foot[0]), (255, 0, 0), 2)
    mask_and_mat = np.concatenate((mask, yoga_mat_mask), axis=1)
    cv2.imshow("Frame", mask_and_mat)
    key = cv2.waitKey(0)
    if key == 27 or key == ord('q'):
        break

cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)
    

Found top black pixel at [ 50 312]
Found top black pixel at [ 50 317]
Found top black pixel at [ 50 312]
Found top black pixel at [ 50 314]
Found top black pixel at [ 52 309]
Found top black pixel at [ 52 309]
Found top black pixel at [ 50 312]
Found top black pixel at [ 50 312]
Found top black pixel at [ 50 312]
Found top black pixel at [ 50 312]
Found top black pixel at [ 50 312]
Found top black pixel at [ 50 312]
Found top black pixel at [ 50 312]
Found top black pixel at [ 50 311]
Found top black pixel at [ 50 312]
Found top black pixel at [ 50 312]
Found top black pixel at [ 50 312]
Found top black pixel at [ 50 317]
Found top black pixel at [ 50 317]
Found top black pixel at [ 50 314]
Found top black pixel at [ 50 313]
Found top black pixel at [ 50 314]
Found top black pixel at [ 50 322]
Found top black pixel at [ 50 313]
Found top black pixel at [ 50 312]
Found top black pixel at [ 50 313]
Found top black pixel at [ 50 314]
Found top black pixel at [ 50 314]
Found top black pixe

-1

: 

# Optical Flow Test

In [9]:
# Use Optical Flow to track joints
import cv2
import numpy as np
import matplotlib.pyplot as plt

CLIP_PATH = "/Users/guyperets/Documents/IP_Course/YogaCoach/adav_example_plank.mov"
NUM_JOINTS = 6


In [10]:
frames = load_frames(CLIP_PATH)

In [26]:
joint_coords = []
for i, frame in enumerate(frames):
    if i == 0: # FIRST FRAME
        for _ in range(NUM_JOINTS):
            roi = cv2.selectROI("Frame", frame)
            cv2.waitKey(1)
            cv2.destroyAllWindows()
            cv2.waitKey(1)
            x, y, w, h = roi
            joint_coord = (x + w // 2, y + h // 2) # Center of the ROI
            joint_coords.append(joint_coord)
        joint_coords = np.array(joint_coords, dtype=np.float32)
        prev_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # prev_frame = cv2.GaussianBlur(prev_frame, (3, 3), 0)
        # prev_frame = cv2.Canny(prev_frame, 80, 240)
        continue
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # dtype is uint8
    # frame = cv2.GaussianBlur(frame, (3, 3), 0)
    # frame = cv2.Canny(frame, 80, 240)
    next_pts, status, err = cv2.calcOpticalFlowPyrLK(prev_frame, frame, joint_coords, None)
    status = status.flatten()
    good_new = next_pts[status == 1]
    good_old = joint_coords[status == 1]

    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        # Turn to integers
        a, b, c, d = int(a), int(b), int(c), int(d)
        cv2.circle(frame, (a, b), 5, (255, 0, 0), -1)
        cv2.line(frame, (a, b), (c, d), (255, 0, 0), 2)
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(0)
    if key == 27 or key == ord('q'):
        break
    prev_frame = frame.copy()
        

cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)

Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!


-1

# WORKING CODE (OPTICAL FLOW CODE)

In [14]:
import cv2
import numpy as np


def detect_person(frame, backSub, persistent_mask):
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))

    height, width = frame.shape[:2]
    roi = frame[int(height * 0.1):int(height * 0.9), int(width * 0.25):int(width * 0.75)]

    fg_mask = backSub.apply(roi)

    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, kernel)

    edges = cv2.Canny(roi, 50, 150)
    combined_mask = cv2.bitwise_or(fg_mask, edges)

    # Perform additional morphological operations to connect close components
    combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_CLOSE, kernel, iterations=3)

    # Threshold the mask to remove small noises
    _, combined_mask = cv2.threshold(combined_mask, 127, 255, cv2.THRESH_BINARY)

    # If this is the first frame, initialize the persistent mask
    if persistent_mask is None:
        persistent_mask = combined_mask
    else:
        # Update the persistent mask selectively
        persistent_mask = cv2.addWeighted(persistent_mask, 0.2, combined_mask, 0.8, 0)

        # Apply threshold to ensure persistent mask doesn't accumulate too much information
        _, persistent_mask = cv2.threshold(persistent_mask, 127, 255, cv2.THRESH_BINARY)

    # Find contours in the persistent mask
    contours, _ = cv2.findContours(persistent_mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create an empty mask where the contours will be drawn and filled
    human_mask = np.zeros_like(persistent_mask)

    # Draw and fill the contours on the mask
    cv2.drawContours(human_mask, contours, -1, (255), thickness=cv2.FILLED)

    # Resize the mask back to the original frame size
    full_mask = np.zeros_like(frame)
    full_mask[int(height * 0.1):int(height * 0.9), int(width * 0.25):int(width * 0.75)] = human_mask

    return full_mask, backSub, persistent_mask

In [15]:
import cv2 

def calibration():
    cap = cv2.VideoCapture(0)
    ret, first_frame = cap.read()
    if not ret:
        print("Failed to capture frame.")
        return
    first_frame = cv2.flip(first_frame, 1)
    width = first_frame.shape[1]
    height = first_frame.shape[0]
    w = 70
    h = 70
    joints = {"shoulder": (width // 2 - w // 2, height // 5, w, h), # (x, y, w, h) - xy is the top left corner
              "elbow": (int(width // 2 + 1 * w), height // 5, w, h),
              "wrist": (int(width // 2 + 2 * w), height // 5, w, h),
              "hip": (width // 2 - w // 2, int(height // 5 + 2.75 * h), w, h),
              "knee": (width // 2 - w // 2, int(height // 5 + 4.5 * h), w, h),
              "ankle": (width // 2 - w // 2, int(height // 5 + 5.5 * h + 50), w, h)}
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame.")
            break
        frame = cv2.flip(frame, 1)
        for _, rect in joints.items():
            x, y, w, h = rect
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(1) # When user is ready (fits his joints in the boxes), press space
        if key == 32: # Space ASCII code
            break
    cap.release()
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    centers = {joint: (rect[0] + rect[2] // 2, rect[1] + rect[3] // 2) for joint, rect in joints.items()}
    # print(f"Centers Are: {centers}")
    return centers

In [4]:
import cv2 
import numpy as np

def detect_body_color(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lower = np.array([0, 48, 80])
    upper = np.array([20, 255, 255])
    mask = cv2.inRange(hsv, lower, upper)
    return mask

cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame.")
        break
    frame = cv2.flip(frame, 1)
    body_mask = detect_body_color(frame)
    cv2.imshow("Frame", body_mask)
    cv2.imshow("Original", frame)
    key = cv2.waitKey(1)
    if key == 27 or key == ord('q'):
        break
    
cap.release()
cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)

2024-08-23 14:04:24.194 Python[31216:3329710] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.


Failed to capture frame.


-1

In [1]:
import cv2 as cv
import numpy as np
# Create Kalman Filters for each point in p0
def create_kalman_filters(p0):
    kalman_filters = []
    for point in p0:
        kf = cv.KalmanFilter(4, 2)  # 4 dynamic params (x, y, vx, vy), 2 measurement params (x, y)
        kf.transitionMatrix = np.array([[1, 0, 1, 0],
                                        [0, 1, 0, 1],
                                        [0, 0, 1, 0],
                                        [0, 0, 0, 1]], np.float32)
        kf.measurementMatrix = np.eye(2, 4, dtype=np.float32)
        kf.processNoiseCov = np.eye(4, 4, dtype=np.float32) * 0.03
        kf.measurementNoiseCov = np.eye(2, 2, dtype=np.float32) * 1
        kf.errorCovPost = np.eye(4, 4, dtype=np.float32) * 0.1
        kf.statePost[:2, 0] = point.ravel()  # Initial position
        kalman_filters.append(kf)
    return kalman_filters

In [18]:
import numpy as np
import cv2 as cv


JOINTS_LOCATIONS = calibration() # Returns a dictionary with the {joint: joint_coords}
#CLIP_PATH = "/Users/guyperets/Documents/IP_Course/YogaCoach/adva_example_chaturanga.mov"
# frames = load_frames(CLIP_PATH)

backSub = cv.createBackgroundSubtractorMOG2(history=100, varThreshold=50, detectShadows=False)
persistent_mask = None
# Parameters for Lucas-Kanade optical flow
lk_params = {
             "winSize" : (15, 15),
             "maxLevel": 3,
             "criteria": (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03),
             "minEigThreshold": 1e-4,
             "flags": 0
            }
                  

# Create some random colors
color = np.random.randint(0, 255, (100, 3))

# Take first frame and find corners in it
cap = cv.VideoCapture(0)
ret, old_frame = cap.read()
if not ret:
    print("Failed to capture frame.")
    exit()
joint_coords = list(JOINTS_LOCATIONS.values())
old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
old_gray = cv.GaussianBlur(old_gray, (3, 3), 0)
# segmented_old, backSub, persistent_mask = detect_person(old_gray, backSub, persistent_mask)
# for _ in range(NUM_JOINTS): # ! REPLACED WITH BOXES CALIBRATION
#     roi = cv2.selectROI("Frame", old_frame)
#     cv2.waitKey(1)
#     cv2.destroyAllWindows()
#     cv2.waitKey(1)
#     x, y, w, h = roi
#     joint_coord = (x + w // 2, y + h // 2) # Center of the ROI
#     joint_coords.append(joint_coord)

fade_factor = 0.6  # Factor by which to fade the mask in each iteration
fade_threshold = 10  # Intensity below which lines will be completely removed

p0 = np.array(joint_coords, dtype=np.float32)
kfs = create_kalman_filters(p0)
# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)
count = 0
hsv = np.zeros_like(old_gray)
hsv[..., 1] = 255
while True:
    ret, frame = cap.read()
    if not ret:
        print("ERROR: Failed To Capture Frame")
        exit()
    frame = cv2.flip(frame, 1)
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    frame_gray = cv.GaussianBlur(frame_gray, (3, 3), 0)
    # segmented_frame, backSub, persistent_mask = detect_person(frame_gray, backSub, persistent_mask)
    predicted_points = []
    for kf in kfs:
        prediction = kf.predict()
        predicted_points.append(prediction[:2].reshape(-1))

    predicted_points = np.array(predicted_points, dtype=np.float32).reshape(-1, 1, 2)
    # calculate optical flow
    p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Select good points
    if p1 is not None and st is not None:
        good_new = []
        for i, kf in enumerate(kfs):
            if st[i]:
                measurement = p1[i].reshape(-1, 1)
                kf.correct(measurement)
                good_new.append(p1[i].reshape(2))
            else:
                if count < 100:
                    count += 1
                    good_new.append(p0[i].reshape(2))
                else:
                    good_new.append(predicted_points[i].reshape(2))
    else:
        print("ERROR: Optical flow calculation failed")
        p0 = np.array(joint_coords, dtype=np.float32)
        old_gray = frame_gray.copy()
        continue
    good_new = np.array(good_new, dtype=np.float32).reshape(-1, 1, 2)
    # draw the tracks
    for i, (new, old) in enumerate(zip(good_new, p0)):
        a, b = new.ravel()
        c, d = old.ravel()
        a, b, c, d = int(a), int(b), int(c), int(d)
        cv.line(mask, (a, b), (c, d), color[i].tolist(), 2)
        cv.circle(frame, (a, b), 5, color[i].tolist(), -1)
    mask = cv.multiply(mask, fade_factor)
    mask[mask < fade_threshold] = 0
    img = cv.add(frame, mask)
    connect_relevant_joints(img, good_new)
    check_pose(img, good_new)
    cv.imshow('frame', img)
    k = cv.waitKey(1)
    if k == 27 or k == ord('q'):
        break

    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

cap.release()
cv2.waitKey(1)
cv.destroyAllWindows()
cv2.waitKey(1)


-1

# Farneback Optical Flow Test

In [2]:
import numpy as np
import cv2 as cv


cap = cv.VideoCapture(0)
ret, frame1 = cap.read()
if not ret:
    print("Failed to capture frame.")
    exit()
prvs = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255
while True:
    ret, frame2 = cap.read()
    if not ret:
        print('Failed to capture frame.')
        continue
    frame2 = cv.flip(frame2, 1)
    frame2 = cv.GaussianBlur(frame2, (3, 3), 0)
    next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang*180/np.pi/2
    hsv[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
    gray = cv.cvtColor(bgr, cv.COLOR_BGR2GRAY)
    THRESHOLD = 70
    gray[gray < THRESHOLD] = 0
    gray[gray >= THRESHOLD] = 255
    cv.imshow('frame2', gray)
    k = cv.waitKey(1)
    if k == 27 or k == ord('q'):
        break
    prvs = next

cv.waitKey(1)
cv.destroyAllWindows()
cv.waitKey(1)

-1

# Visualising The Movement

In [16]:
import numpy as np
import cv2 as cv

# Initialize video capture
cap = cv.VideoCapture(0)
ret, frame1 = cap.read()
if not ret:
    print("Failed to capture frame.")
    exit()

# Convert the first frame to grayscale
prvs = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)

# Parameters for visualizing the flow field
step = 16  # Step size for drawing arrows
h, w = prvs.shape[:2]

# Main loop
while True:
    ret, frame2 = cap.read()
    if not ret:
        print('Failed to capture frame.')
        continue

    frame2 = cv.flip(frame2, 1)
    frame2 = cv.GaussianBlur(frame2, (3, 3), 0)
    next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)

    # Calculate the dense optical flow using Farneback method
    flow = cv.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # Draw the optical flow field
    for y in range(0, h, step):
        for x in range(0, w, step):
            # Get the flow vector at this point
            fx, fy = flow[y, x]
            # Draw an arrow on the image
            cv.arrowedLine(frame2, (x, y), (int(x + fx), int(y + fy)), (0, 255, 0), 2, tipLength=0.3)

    # Display the frame with the movement arrows
    cv.imshow('Movement Field', frame2)

    # Wait for key press
    k = cv.waitKey(1)
    if k == 27 or k == ord('q'):
        break

    # Update previous frame
    prvs = next

# Cleanup
cv.waitKey(1)
cv.destroyAllWindows()
cv.waitKey(1)

-1

In [3]:
JOINT_NAMES = ["WRIST", "SHOULDER", "ELBOW", "HIP", "KNEE", "ANKLE"]
def connect_relevant_joints(img, joints_coords):
    # Connect all joints one after another
    for i in range(len(joints_coords) - 1):
        # Convert to int (pixel coordinates)
        joint1 = tuple(map(int, joints_coords[i][0]))
        joint2 = tuple(map(int, joints_coords[i + 1][0]))
        # Draw the line
        cv.line(img, joint1, joint2, (0, 255, 0), 2)

In [4]:
pose_params = [("SHOULDER", "ELBOW", "WRIST", 180), ("SHOULDER", "HIP", "ANKLE", 120)]
def check_pose(img, joints_coords):
    for i, (joint1, joint2, joint3, angle) in enumerate(pose_params):
        joint1_idx = JOINT_NAMES.index(joint1)
        joint2_idx = JOINT_NAMES.index(joint2)
        joint3_idx = JOINT_NAMES.index(joint3)
        angle = np.deg2rad(angle)
        joint1_coords = joints_coords[joint1_idx].ravel()
        joint2_coords = joints_coords[joint2_idx].ravel()
        joint3_coords = joints_coords[joint3_idx].ravel()
        v1 = joint1_coords - joint2_coords
        v2 = joint3_coords - joint2_coords
        cos_angle = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
        angle_between_joints = np.arccos(cos_angle)
        text_position = (10, 30 * (i * 2 + 1))
        if abs(angle_between_joints - angle) < 5:
            cv.putText(  # TODO CHANGE TO TTS
                img,
                f"Angle between {joint1}, {joint2} and {joint3} is {np.rad2deg(angle_between_joints):.2f} degrees",
                text_position,
                cv.FONT_HERSHEY_SIMPLEX,
                1,
                (0, 255, 0),
                2,
                cv.LINE_AA,
            )
        else:
            cv.putText(  # TODO CHANGE TO TTS
                img,
                "FIX YOUR POSE",
                text_position,
                cv.FONT_HERSHEY_SIMPLEX,
                1,
                (0, 0, 255),
                2,
                cv.LINE_AA,
            )

# FULL TEST

In [ ]:
# FOR ADVA FULL VIDEO
# POSE CORRESPONDING FRAME INDEXES ARE
PLANK_INDEXES = [90, 120]
CHITURANGA_INDEXES = [150, 180]
UPWARD_DOG_INDEXES = [200, 260]
DOWNWARD_DOG_INDEXES = [330, 500]

In [6]:
import numpy as np
import cv2 as cv


CLIP_PATH = "/Users/guyperets/Documents/IP_Course/Project/DEMO_FULL.mov"
frames = load_frames(CLIP_PATH)
static_frames = 0
backSub = cv.createBackgroundSubtractorMOG2(history=100, varThreshold=50, detectShadows=False)
persistent_mask = None
# Parameters for Lucas-Kanade optical flow
lk_params = {
             "winSize" : (15, 15),
             "maxLevel": 3,
             "criteria": (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03),
             "minEigThreshold": 1e-4,
             "flags": 0
            }
                  
# Create some random colors
color = np.random.randint(0, 255, (100, 3))

# Take first frame and find corners in it
old_frame = frames[0]
curr_frame_idx = 1
joint_coords = []
old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)
old_gray = cv.GaussianBlur(old_gray, (3, 3), 0)
for _ in range(6): # ! REPLACED WITH BOXES CALIBRATION
    roi = cv2.selectROI("Frame", old_frame)
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    x, y, w, h = roi
    joint_coord = (x + w // 2, y + h // 2) # Center of the ROI
    joint_coords.append(joint_coord)

fade_factor = 0.6  # Factor by which to fade the mask in each iteration
fade_threshold = 10  # Intensity below which lines will be completely removed

p0 = np.array(joint_coords, dtype=np.float32)
kfs = create_kalman_filters(p0)
# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)
count = 0
hsv = np.zeros_like(old_gray)
hsv[..., 1] = 255
while True:
    frame = frames[curr_frame_idx]
    curr_frame_idx += 1
    if curr_frame_idx == len(frames):
        break
    # frame = cv2.flip(frame, 1)
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    frame_gray = cv.GaussianBlur(frame_gray, (3, 3), 0)
    # segmented_frame, backSub, persistent_mask = detect_person(frame_gray, backSub, persistent_mask)
    predicted_points = []
    for kf in kfs:
        prediction = kf.predict()
        predicted_points.append(prediction[:2].reshape(-1))

    predicted_points = np.array(predicted_points, dtype=np.float32).reshape(-1, 1, 2)
    # calculate optical flow
    p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    # Select good points
    if p1 is not None and st is not None:
        good_new = []
        for i, kf in enumerate(kfs):
            if st[i]:
                measurement = p1[i].reshape(-1, 1)
                kf.correct(measurement)
                good_new.append(p1[i].reshape(2))
            else:
                if count < 100:
                    count += 1
                    good_new.append(p0[i].reshape(2))
                else:
                    good_new.append(predicted_points[i].reshape(2))
    else:
        print("ERROR: Optical flow calculation failed")
        p0 = np.array(joint_coords, dtype=np.float32)
        old_gray = frame_gray.copy()
        continue
    good_new = np.array(good_new, dtype=np.float32).reshape(-1, 1, 2)
    # draw the tracks
    for i, (new, old) in enumerate(zip(good_new, p0)):
        a, b = new.ravel()
        c, d = old.ravel()
        a, b, c, d = int(a), int(b), int(c), int(d)
        cv.line(mask, (a, b), (c, d), color[i].tolist(), 2)
        cv.circle(frame, (a, b), 5, color[i].tolist(), -1)
    mask = cv.multiply(mask, fade_factor)
    mask[mask < fade_threshold] = 0
    img = cv.add(frame, mask)
    connect_relevant_joints(img, good_new)
    # Start checking the pose only after there is no much movement (the pose should static)
    if np.all(p1 - p0 <= 2):
        static_frames += 1
    else:
        static_frames = 0
    if static_frames > 10:
        check_pose(img, good_new)
    
    cv.putText(img, f"Frame: {curr_frame_idx}", (1100, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv.LINE_AA)
    cv.imshow('frame', img)
    k = cv.waitKey(100)
    if k == 27 or k == ord('q'):
        break

    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

cv2.waitKey(1)
cv.destroyAllWindows()
cv2.waitKey(1)


Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!


-1

In [1]:
from gtts import gTTS
import os

# Text to convert to speech
text = "Hello, this is a test of Google Text-to-Speech."

# Create a gTTS object
tts = gTTS(text=text, lang='en')

# Play the speech
tts.save("hello.mp3")
os.system("mpg321 hello.mp3")

High Performance MPEG 1.0/2.0/2.5 Audio Player for Layer 1, 2, and 3.
Version 0.3.2-1 (2012/03/25). Written and copyrights by Joe Drew,
now maintained by Nanakos Chrysostomos and others.
Uses code from various people. See 'README' for more!
THIS SOFTWARE COMES WITH ABSOLUTELY NO WARRANTY! USE AT YOUR OWN RISK!
tcgetattr(): Operation not supported on socket

Playing MPEG stream from hello.mp3 ...
MPEG 2.0 layer III, 64 kbit/s, 24000 Hz mono
                                                                            
[0:03] Decoding of hello.mp3 finished.


0

In [1]:
import cv2

def get_pixel_coordinates(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:  # If the left mouse button was clicked
        return x, y

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open video stream.")
    exit()

cv2.namedWindow("Calibration Frame")
cv2.setMouseCallback("Calibration Frame", get_pixel_coordinates)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Display the frame
    cv2.imshow("Video Frame", frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close all OpenCV windows
cap.release()
cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)

Mouse clicked at position: (711, 180)
Mouse clicked at position: (395, 529)


KeyboardInterrupt: 

: 

In [1]:
import cv2

# Initial text to display
text = "Click anywhere on the screen"

# Callback function to capture mouse events and update text
def get_pixel_coordinates(event, x, y, flags, param):
    global text
    if event == cv2.EVENT_LBUTTONDOWN:  # If the left mouse button was clicked
        text = f"Last click at: ({x}, {y})"
        print(text)  # Print the coordinates to the console as well

# Start capturing video from the webcam
cap = cv2.VideoCapture(0)

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not open video stream.")
    exit()

# Set up a named window and bind the mouse callback function to this window
cv2.namedWindow("Video Frame")
cv2.setMouseCallback("Video Frame", get_pixel_coordinates)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Display the text on the frame
    cv2.putText(frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    # Show the frame
    cv2.imshow("Video Frame", frame)

    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close all OpenCV windows
cap.release()
cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)

Last click at: (769, 248)
Last click at: (970, 276)
Last click at: (897, 407)
Last click at: (507, 517)
Last click at: (654, 277)
Last click at: (805, 362)
Last click at: (763, 204)
Last click at: (763, 203)


KeyboardInterrupt: 

: 

In [4]:
from gtts import gTTS
from pydub import AudioSegment
import os

# Text to convert to speech
text = "Hello, this is a test of Google Text-to-Speech."

# Create a gTTS object
tts = gTTS(text=text, lang='en')

# Save the speech to an MP3 file
tts.save("hello.mp3")

# Load the MP3 file
sound = AudioSegment.from_file("hello.mp3")

# Speed up the audio (1.5x)
faster_sound = sound.speedup(playback_speed=1.2)

# Save the modified file
faster_sound.export("hello_faster.mp3", format="mp3")

# Play the modified file
os.system("mpg321 hello_faster.mp3")

High Performance MPEG 1.0/2.0/2.5 Audio Player for Layer 1, 2, and 3.
Version 0.3.2-1 (2012/03/25). Written and copyrights by Joe Drew,
now maintained by Nanakos Chrysostomos and others.
Uses code from various people. See 'README' for more!
THIS SOFTWARE COMES WITH ABSOLUTELY NO WARRANTY! USE AT YOUR OWN RISK!
tcgetattr(): Operation not supported on socket

Playing MPEG stream from hello_faster.mp3 ...
MPEG 2.0 layer III, 64 kbit/s, 24000 Hz mono
                                                                            
[0:02] Decoding of hello_faster.mp3 finished.


0